In [ ]:
import os
import math
import shutil
import warnings
import importlib
import urllib.request as request
from contextlib import closing
from numba import cuda

import cupy
import cudf
import cuml

import dask_cudf
import dask_ml
import dask

from cuml.manifold import UMAP as cuUMAP
from cuml.dask.decomposition import PCA as cuDaskPCA
from cuml.dask.cluster import KMeans as cuDaskKMeans
from cuml.dask.manifold import UMAP as cuDaskUMAP

from dask.distributed import Client, LocalCluster
from dask_cuda import initialize, LocalCUDACluster
from dask_cuda.local_cuda_cluster import cuda_visible_devices

from bokeh.io.export import export_png
from bokeh.plotting import figure
from bokeh.models.tickers import FixedTicker
from bokeh.io import output_notebook, push_notebook, show

import nvidia.cheminformatics.chembldata as chembldata

warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')

output_notebook()

### Configurations and settings

In [ ]:
# Please add or remove device ids that can be used
# CUDA_VISIBLE_DEVICES=[0]
CUDA_VISIBLE_DEVICES = cuda_visible_devices(0).split(',')

pca_comps = 64
n_clusters = 6
n_neighbors=100
num_mols=5000

enable_tcp_over_ucx = True
enable_nvlink = False
enable_infiniband = False

COLORS = ["#406278", "#e32636", "#9966cc", "#cd9575", "#915c83", "#008000",
          "#ff9966", "#848482", "#8a2be2", "#de5d83", "#800020", "#e97451",
          "#5f9ea0", "#36454f", "#008b8b", "#e9692c", "#f0b98d", "#ef9708",
          "#0fcfc0", "#9cded6", "#d5eae7", "#f3e1eb", "#f6c4e1", "#f79cd4"]
FINGER_PRINT_FILES = 'filter_*.h5'

# Download ChEMBL database

In [ ]:
data_dir = "/data/db"
db_file = os.path.join(data_dir, 'chembl_27.db')

if not os.path.exists(db_file):
    print('Downloading ChEMBL db...')

    os.makedirs(data_dir, exist_ok=True)
    with closing(request.urlopen('ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_27_sqlite.tar.gz')) as r:
        with open(db_file, 'wb') as f:
            shutil.copyfileobj(r, f)

    print('Download completed')
else:
    print('Reusing available ChEMBL db at', db_file)

In [4]:
cluster = LocalCUDACluster(protocol="ucx",
                           dashboard_address=':9001',
                           # TODO: automate visible device list
                           CUDA_VISIBLE_DEVICES=CUDA_VISIBLE_DEVICES,
                           enable_tcp_over_ucx=enable_tcp_over_ucx,
                           enable_nvlink=enable_nvlink,
                           enable_infiniband=enable_infiniband)
client = Client(cluster)
n_workers = len(client.scheduler_info()['workers'].keys())
client

Client Scheduler: ucx://127.0.0.1:54643 Dashboard: http://127.0.0.1:9001/status,Cluster Workers: 1 Cores: 1 Memory: 49.27 GB


# Generate fingerprint from ChEMBL

The 4 in ECFP4 corresponds to the diameter of the atom environments considered, while the Morgan fingerprints take a radius parameter. So a Morgan fingerprint with radius=2 is roughly equivalent to ECFP4 and FCFP4.

In [5]:
%%time
import nvidia.cheminformatics.chembldata
importlib.reload(nvidia.cheminformatics.chembldata)

cache_directory = '/data/fingerprint/'
# cache_directory = None
from nvidia.cheminformatics.fingerprint import MorganFingerprint

if cache_directory is None:
    chem_data = chembldata.ChEmblData(db_file=db_file, fp_type=MorganFingerprint)
    ddf = chem_data.fetch_all_props(num_recs=num_mols)
else:
    hdf_path = os.path.join(cache_directory, FINGER_PRINT_FILES)
    ddf = dask.dataframe.read_hdf(hdf_path, 'fingerprints')

    if num_mols > 0:
        ddf = ddf.head(num_mols, compute=False, npartitions=-1)

dcudf = dask_cudf.from_dask_dataframe(ddf)
dcudf = dcudf.persist()
df = dcudf.compute()

CPU times: user 12.9 s, sys: 1.37 s, total: 14.3 s
Wall time: 47.1 s


## Tanimoto Similarity

In [8]:
# Implementation of https://en.wikipedia.org/wiki/Jaccard_index#Other_definitions_of_Tanimoto_distance

@cuda.jit
def compute_norms(data, norms):
    i = cuda.grid(1)
    norms[i] = len(data[i])
    for j in range(len(data[i])):
        if data[i][j] != 0:
            value = j + 1
            data[i][j] = value
            norms[i] = norms[i] + (value**2)
    
    if norms[i] != 0:
        norms[i] = math.sqrt(norms[i])

@cuda.jit
def compute_tanimoto_sim_matix(data, norms, sim_array):
    x = cuda.grid(1)
    rows = len(data)
    
    i = x // rows
    j = x % rows
    
    if i == j:
        sim_array[i][j] = 1
        return
    
    a = data[i]
    b = data[j]
    
    prod = 0
    for k in range(len(data[i])):
        prod = prod + (a[k] * b[k])
        
    a_norm = norms[i]
    b_norm = norms[j]
    
    sim_array[i][j] = (prod / ((a_norm**2 + b_norm**2) - prod))
    
def tanimotoSimilarity(fp):
    norms = cupy.zeros(fp.shape[0])
    compute_norms.forall(norms.shape[0], 1)(fp, norms)

    sim_array = cupy.zeros((fp.shape[0], fp.shape[0]), cupy.float32)
    compute_tanimoto_sim_matix.forall(fp.shape[0] * fp.shape[0], 1)(fp, norms, sim_array)
    return sim_array

In [9]:
kmeans_float = cuml.KMeans(n_clusters=n_clusters, 
                           random_state=0,
                          init='k-means||')

fp = cupy.fromDlpack(df.to_dlpack())
sim_array = tanimotoSimilarity(fp)
df2 = cudf.DataFrame(sim_array)
kmeans_float.fit(df2)
clusters = kmeans_float.labels_

## Calculate Silhouette Score in Batches

In [10]:
import math
import cudf
import pandas
import numpy
from sklearn.metrics import silhouette_score

def batched_silhouette_scores(embeddings, clusters, batch_size=5000, seed=0, on_gpu=True):
    """Calculate silhouette score in batches on the CPU

    Args:
        embeddings (cudf.DataFrame or cupy.ndarray): input features to clustering
        clusters (cudf.DataFrame or cupy.ndarray): cluster values for each data point
        batch_size (int, optional): Size for batching. Defaults to 5000.
        seed (int, optional): Random seed. Defaults to 0.
        on_gpu (bool, optional): Input data is on GPU. Defaults to True.

    Returns:
        float: mean silhouette score from batches
    """

    if on_gpu:
        arraylib = cupy
        dflib = cudf
        AsArray = cupy.asnumpy
    else:
        arraylib = numpy
        dflib = pandas
        AsArray = numpy.asarray

    # Function to calculate results
    def _silhouette_scores(input_data):
        embeddings, clusters = input_data
        return silhouette_score(AsArray(embeddings), AsArray(clusters))

    # Shuffle on GPU
    combined = dflib.DataFrame(embeddings) if not isinstance(embeddings, dflib.DataFrame) else embeddings
    embeddings_columns = combined.columns
    cluster_column = 'clusters'

    clusters = dflib.Series(clusters, name=cluster_column)
    combined[cluster_column] = clusters
    combined = combined.sample(n=len(combined), replace=False, random_state=seed) # shuffle via sampling

    embeddings = combined[embeddings_columns]
    clusters = combined[cluster_column]

    # Chunk arrays
    if on_gpu:
        embeddings = cupy.fromDlpack(embeddings.to_dlpack())
        clusters = cupy.fromDlpack(clusters.to_dlpack())

    n_chunks = int(math.ceil(len(embeddings) / batch_size))
    embeddings_chunked = arraylib.array_split(embeddings, n_chunks)
    clusters_chunked = arraylib.array_split(clusters, n_chunks)
    
    # Calculate scores on batches and return the average
    scores = list(map(_silhouette_scores, zip(embeddings_chunked, clusters_chunked)))
    return numpy.array(scores).mean()

In [13]:
# GPU version
df2 = cudf.DataFrame(sim_array)
clusters = kmeans_float.labels_
on_gpu = True

batched_silhouette_scores(df2, clusters, batch_size=300, on_gpu=on_gpu)

0.07745108

In [12]:
# CPU version
on_gpu = False
batched_silhouette_scores(df2.to_pandas(), clusters.to_pandas(), batch_size=300, on_gpu=on_gpu)

0.07859744